In [35]:
%pip install transformers
%pip install 'accelerate>=0.26.0'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'accelerate"


In [39]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-32B-Instruct"

# Überprüfen, ob CUDA verfügbar ist und das Gerät entsprechend setzen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print("CUDA is available. Using GPU.")
else:
    print("CUDA is not available. Using CPU.")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)  # Modell auf das entsprechende Gerät verschieben

tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Eingaben auf das entsprechende Gerät verschieben
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

CUDA is available. Using GPU.


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [22]:
import argparse
import re
import time
import torch
from transformers import pipeline, RobertaTokenizer, RobertaForCausalLM

In [28]:
device = 0 if torch.cuda.is_available() else -1
if device == 0:
    print("CUDA is available. Using GPU.")
else:
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [32]:
def create_prompt(question):
    """ Generate prompt to translate question to SQL query.
     
    Args:
        question: question about data in natural language.
    
    Returns:
        prompt for question translation.
    """

    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\textbackslash{}n'.join(parts)

def call_llm(prompt):
    """ Query large language model and return answers.
    
    Args:
        prompt: input prompt for large language model.
    
    Returns:
        Answer by language model.
    """

    nlp = pipeline("text-generation", model="roberta-base", device=device, is_decoder=True)
    for nr_retries in range(1, 4):
        try:
            response = nlp(prompt, max_length=150, truncation=True)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Attempt {nr_retries} failed: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query roberta-base!')
    
 
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='Question about games.')
    args = parser.parse_args(['How many games are there in the database?'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt)

    query = re.findall(r'```sql(.*?)```', answer, re.DOTALL)[0]

    print(f'SQL: {query}')

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


Attempt 1 failed: The following `model_kwargs` are not used by the model: ['is_decoder'] (note: typos in the generate arguments will also show up in this list)
Attempt 2 failed: The following `model_kwargs` are not used by the model: ['is_decoder'] (note: typos in the generate arguments will also show up in this list)
Attempt 3 failed: The following `model_kwargs` are not used by the model: ['is_decoder'] (note: typos in the generate arguments will also show up in this list)


Exception: Cannot query roberta-base!